In [ ]:
#register a model without using an experiment
model = Model.register(workspace = ws,
                        model_path ="mnist/model.onnx",
                        model_name = "onnx_mnist",
                        tags = {"onnx": "demo"},
                        description = "MNIST image classification CNN from ONNX Model Zoo",)

The following example demonstrates loading an environment from your workspace and then using it with the inference configuration:


Define an Inference Configuration
The model will be deployed as a service that consist of:

A script to load the model and return predictions for submitted data.
An environment in which the script will be run.

You must therefore define the script and environment for the service.

Creating an Entry Script
Create the entry script (sometimes referred to as the scoring script) for the service as a Python (.py) file. It must include two functions:

init(): Called when the service is initialized.
run(raw_data): Called when new data is submitted to the service.
Typically, you use the init function to load the model from the model registry, and use the run function to generate predictions from the input data. The following example script shows this pattern:


In [ ]:
# score.py
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the registered model file and load it
    model_path = Model.get_model_path('classification_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Return the predictions as any JSON serializable format
    return predictions.tolist()

Creating an Environment
Your service requires a Python environment in which to run the entry script, which you can configure using Conda configuration file. An easy way to create this file is to use a CondaDependencies class to create a default environment (which includes the azureml-defaults package and commonly-used packages like numpy and pandas), add any other required packages, and then serialize the environment to a string and save it:

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

# Add the dependencies for your model
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = 'service_files/env.yml'
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

Combining the Script and Environment in an InferenceConfig
After creating the entry script and environment configuration file, you can combine them in an InferenceConfig for the service like this:

In [2]:
from azureml.core.model import InferenceConfig

classifier_inference_config = InferenceConfig(runtime= "python",
                                              source_directory = 'service_files',
                                              entry_script="score.py",
                                              conda_file="env.yml")

ERROR - source directory /mnt/batch/tasks/shared/LS_root/mounts/clusters/jupyter-compute/code/git/tmpmlops/jupyter-notebooks/service_files doesn't exist. 



WebserviceException: WebserviceException:
	Message: source directory /mnt/batch/tasks/shared/LS_root/mounts/clusters/jupyter-compute/code/git/tmpmlops/jupyter-notebooks/service_files doesn't exist. 
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "source directory /mnt/batch/tasks/shared/LS_root/mounts/clusters/jupyter-compute/code/git/tmpmlops/jupyter-notebooks/service_files doesn't exist. "
    }
}

In [ ]:
#After creating the entry script and environment configuration file, you can combine them in an InferenceConfig for the service like this:
from azureml.core.model import InferenceConfig

classifier_inference_config = InferenceConfig(runtime= "python",
                                              source_directory = 'service_files',
                                              entry_script="score.py",
                                              conda_file="env.yml")

In [ ]:
# from azureml.core.environment import Environment
# myenv = Environment.from_conda_specification(name = 'myenv',
#                                                 file_path = 'path-to-conda-specification-file'
# myenv.register(workspace=ws)

In [ ]:
# from azureml.core.environment import Environment
# from azureml.core.model import InferenceConfig


# myenv = Environment.get(workspace=ws, name='myenv', version='1')
# inference_config = InferenceConfig(entry_script='path-to-score.py',
#                                     environment=myenv)

In [ ]:
# #deployment
# from azureml.core.webservice import AciWebservice, AksWebservice, LocalWebservice

# deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

# service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config)
# service.wait_for_deployment(show_output = True)
# print(service.state)

Deployment


In [ ]:
# create the compute resource
from azureml.core.compute import ComputeTarget, AksCompute

cluster_name = 'aks-cluster'
compute_config = AksCompute.provisioning_configuration(location='eastus')
production_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
production_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.core.webservice import AksWebservice

classifier_deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 1)


from azureml.core.model import Model

model = ws.models['classification_model']
service = Model.deploy(workspace=ws,
                       name = 'classifier-service',
                       models = [model],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config,
                       deployment_target = production_cluster)
service.wait_for_deployment(show_output = True)